In [77]:
from dotenv import load_dotenv
import os
load_dotenv()  # .env 파일 로드

embedding_api_key = os.getenv("Embedding_API_KEY")
embedding_endpoint = os.getenv("Embedding_ENDPOINT")

gpt_api_key = os.getenv('OPENAI_API_KEY')
gpt_endpoint = os.getenv('OPENAI_ENDPOINT')

BLOB_CONN_STR = os.getenv('BLOB_CONN_STR')
DI_ENDPOINT = os.getenv('DI_ENDPOINT')
DI_API_KEY = os.getenv('DI_API_KEY')



### PDF-> 마크다운 형태로 변환

In [83]:
import os
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from azure.core.credentials import AzureKeyCredential
from datetime import datetime, timedelta

# ✅ 1. 설정
BLOB_CONTAINER_NAME = "pdf-container"
PDF_LOCAL_PATH = "data/pdf/LH-25년1차청년매입임대입주자모집공고문(서울지역본부).pdf"
PDF_BLOB_NAME = "uploaded_test.pdf"


# ✅ 2. Blob Storage에 업로드
blob_service = BlobServiceClient.from_connection_string(BLOB_CONN_STR)
container_client = blob_service.get_container_client(BLOB_CONTAINER_NAME)

# 컨테이너가 없다면 생성
if not container_client.exists():
    container_client.create_container()

blob_client = container_client.get_blob_client(PDF_BLOB_NAME)

# PDF 파일 업로드
with open(PDF_LOCAL_PATH, "rb") as f:
    blob_client.upload_blob(f, overwrite=True)
print("✅ PDF 업로드 완료")

# ✅ 3. SAS URL 생성 (15분 유효)
sas_token = generate_blob_sas(
    account_name=blob_service.account_name,
    container_name=BLOB_CONTAINER_NAME,
    blob_name=PDF_BLOB_NAME,
    account_key=blob_service.credential.account_key,
    permission=BlobSasPermissions(read=True),
    expiry=datetime.utcnow() + timedelta(minutes=15)
)

sas_url = f"{blob_client.url}?{sas_token}"
print("✅ SAS URL 생성 완료")

# ✅ 4. Document Intelligence Markdown 분석
client = DocumentIntelligenceClient(endpoint=DI_ENDPOINT, credential=AzureKeyCredential(DI_API_KEY))

poller = client.begin_analyze_document(
    model_id="prebuilt-layout",
    body=AnalyzeDocumentRequest(url_source=sas_url),
    output_content_format='markdown'
)

result = poller.result()

# ✅ 5. 결과 저장
with open("data/markdown/LH-25년1차청년매입임대입주자모집공고문(서울지역본부).md", "w", encoding="utf-8") as f:
    f.write(result.content)
print("✅ Markdown 분석 결과 저장 완료")


✅ PDF 업로드 완료
✅ SAS URL 생성 완료
✅ Markdown 분석 결과 저장 완료


### 문서 로딩

In [84]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

# ✅ Azure OpenAI GPT-4o 호출 함수
def request_gpt(prompt: str) -> str:
    
    headers = {
        'Content-Type': 'application/json',
        'api-key': gpt_api_key
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": (
                    '너는 HTML 테이블을 사람이 이해할 수 있도록 자연어 문장으로 변환해.  '
                    '항목과 값을 "구분: 내용" 식으로 나누지 말고, 원래 테이블에서 쓰인 항목명을 그대로 key로 사용해.  '
                    '예: "임대조건 : 내용", "임대보증금-월임대료 전환 : 내용"처럼.  '
                    '항목이 반복되는 경우에는 구분자를 붙여서 명확하게 구분해줘.  '
                    '불필요한 요약이나 도입부 없이 표의 핵심 내용만 변환해줘.'


                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    response = requests.post(gpt_endpoint, headers=headers, json=body)

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print("❌ 요청 실패:", response.status_code, response.text)
        return "⚠️ 오류가 발생했습니다."


# ✅ 병렬 LLM 요청 + md 파일 내 table 변환
def convert_md_tables_with_llm_parallel(md_file_path, output_path=None, max_workers=5):
    with open(md_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    soup = BeautifulSoup(content, 'html.parser')
    tables = soup.find_all('table')
    table_strs = [str(table) for table in tables]

    # 중복 제거
    unique_tables = list(set(table_strs))
    table_to_text = {}

    # 내부 처리 함수
    def process_table(table_html):
        prompt = (
        f"다음 HTML 테이블의 내용을 자연어 문장으로 간결하게 변환해줘."
        f"항목: 내용 형식으로 출력하되, 항목명은 테이블 안의 텍스트 그대로 사용하고, '구분', '내용' 같은 일반적인 단어로 묶지 말아줘."
        f"동일한 항목이 여러 개 나오면 괄호 등을 써서 구분해줘.\n\n{table_html}"
        )

        result = request_gpt(prompt)
        return table_html, result

    print(f"📊 총 테이블 수: {len(tables)} (고유 테이블: {len(unique_tables)})")
    print(f"🚀 LLM에 병렬 요청 시작 (max_workers={max_workers})")

    # 병렬 처리
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_table, tbl) for tbl in unique_tables]
        for future in as_completed(futures):
            tbl_html, gpt_result = future.result()
            table_to_text[tbl_html] = gpt_result

    # 원문 치환
    for original_table in table_strs:
        if original_table in table_to_text:
            content = content.replace(original_table, table_to_text[original_table])

    # 저장 또는 반환
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✅ 변환 완료! 저장 위치: {output_path}")
    else:
        return content

convert_md_tables_with_llm_parallel(
    md_file_path="data/markdown/LH-25년1차청년매입임대입주자모집공고문(서울지역본부).md",
    output_path="data/markdown/LH-25년1차청년매입임대입주자모집공고문(서울지역본부).md",
    max_workers=5  # 병렬 요청 개수
)


📊 총 테이블 수: 24 (고유 테이블: 24)
🚀 LLM에 병렬 요청 시작 (max_workers=5)
✅ 변환 완료! 저장 위치: data/markdown/LH-25년1차청년매입임대입주자모집공고문(서울지역본부).md


### 마크다운 전처리(':' 처리)

In [85]:
import re

def preprocess_markdown_headers(md_text: str) -> str:
    """
    마크다운 헤더 전처리:
    - ## 제목: 내용 → ## 제목\n내용
    - ■ (제목) 내용 → ■ (제목)\n내용
    """
    # 1. 마크다운 헤더 처리 (## 제목: 내용)
    md_text = re.sub(r'^(#{1,6}\s*■?\s*[^:\n]+):\s*(.+)$', r'\1\n\2', md_text, flags=re.MULTILINE)

    # 2. ■ (제목) 내용 → ■ (제목)\n내용
    md_text = re.sub(r'^(■\s*\([^)]+\))\s+(.+)$', r'\1\n\2', md_text, flags=re.MULTILINE)

    return md_text


In [86]:
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# 2. 전처리
markdown_text_cleaned = preprocess_markdown_headers(markdown_text)

# 3. 분할
splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
    ("#", "section"),
    ("##", "subsection"),
    ("■", "bullet")
])
header_docs = splitter.split_text(markdown_text_cleaned)

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

final_docs = []
for doc in header_docs:
    sub_docs = recursive_splitter.split_text(doc.page_content)
    for chunk in sub_docs:
        final_docs.append(
            Document(page_content=chunk, metadata=doc.metadata)
        )

# 4. 확인 (예시)
for d in final_docs  :
    print("---")
    print("Metadata:", d.metadata)
    print("Content:", d.page_content)


---
Metadata: {}
Content: <!-- PageHeader="살고 싶은 집과 도시로 국민의 희망을 가꾸는 기업" -->  
서울지역본부 청년 매입임대주택 예비입주자 모집공고
[강남구, 강동구, 강북구, 관악구, 구로구, 금천구, 노원구, 동대문구, 마포구, 서대문구, 서초구,
송파구, 용산구, 은평구, 종로구, 중구]  
☐
청년 매입임대주택은 LH에서 주택을 매입하여 청년(만19세~39세), 대학생 및 취업준비생을 대상으로
시중시세 40~50% 수준으로 임대하는 주택입니다.  
☐
LH에서는 마이홈센터(☏1600-1004, 내선번호 2번→3번) 및 서울지역본부 매입임대 상담센터(☏
02-2015-1040)를 통해 모집공고에 대한 안내가 이루어질 수 있도록 상담을 실시하고 있습니다. (다만,
상담내용은 신청 참고자료로만 활용하여 주시고, 반드시 공고문을 통해 세부 내용을 숙지하신 후
신청하여 주시기 바랍니다.). 아울러, 신청자격 미숙지, 착오신청 등에 대해서 신청자 본인에게 책임이 있
---
Metadata: {}
Content: 으니 유의하여 주시기 바랍니다.
□
---
Metadata: {'bullet': '(기준일)'}
Content: 모집공고일은 2025.03.27.(목)이며, 이는 입주자격(신청자격, 나이, 세대구성원, 주택소유,
자산, 소득 등)의 판단기준일이 됩니다.
---
Metadata: {'bullet': '(모집일정)'}
Content: \* 일정은 진행상황에 따라 변동될 수 있습니다.  
모집 공고는 3월 27일(목)에 이루어지며, 신청(PC, 모바일)은 4월 7일(월)부터 4월 9일(수)까지 진행됩니다. 서류심사 대상자 발표는 4월 11일(금)에 있고, 서류 제출은 4월 14일(월)부터 4월 16일(수)까지입니다. 자격 검증은 4월 17일(목)부터 6월 12일(목)까지 이루어지며, 예비 입주자 순번 발표는 6월 13일(금)에 진행됩니다. 계약 체결(순번 도래 시)은 별도의 안내를 따르며, 입주는 계약일부터 60일 이내에

In [87]:
from openai import AzureOpenAI
import numpy as np
import faiss
from langchain.schema import Document
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_openai import AzureOpenAIEmbeddings

# ✅ 필요한 정보


api_version = "2024-02-15-preview"
deployment = "text-embedding-3-small"
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=embedding_endpoint,
    api_key=embedding_api_key
)

# 2. 문서 텍스트 & 메타데이터 준비
texts = [doc.page_content for doc in final_docs]
metadatas = [doc.metadata for doc in final_docs]

# 3. 임베딩 요청
response = client.embeddings.create(
    input=texts,
    model=deployment
)

# 4. 결과 확인 (선택적)
for item, metadata in zip(response.data, metadatas):
    print(f"✅ 청크 {item.index}")
    print(f"→ 임베딩 벡터 길이: {len(item.embedding)}")
    print(f"→ 메타데이터: {metadata}")
    print("=" * 40)


✅ 청크 0
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {}
✅ 청크 1
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {}
✅ 청크 2
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(기준일)'}
✅ 청크 3
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(모집일정)'}
✅ 청크 4
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(중복신청 불가)'}
✅ 청크 5
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(가전제품 등 비치)'}
✅ 청크 6
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(모집단위)'}
✅ 청크 7
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(성별분리)'}
✅ 청크 8
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(모집인원)'}
✅ 청크 9
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(공급방법)'}
✅ 청크 10
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(예비입주자 자격)'}
✅ 청크 11
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(입주지정기간)'}
✅ 청크 12
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(신청유의)'}
✅ 청크 13
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(신청유의)', 'section': '공급개요', 'subsection': '■ 공급대상 주택'}
✅ 청크 14
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(신청유의)', 'section': '공급개요', 'subsection': '■ 임대기간 및 임대조건'}
✅ 청크 15
→ 임베딩 벡터 길이: 1536
→ 메타데이터: {'bullet': '(신청유의)', 'section': '공급개요', 'subsection': '■ 임대기간 

### 로컬에서 테스트

In [90]:
# 5. 문서 + 임베딩 묶기
documents = [
    Document(page_content=text, metadata=meta)
    for text, meta in zip(texts, metadatas)
]
vectors = [item.embedding for item in response.data]
embeddings_np = np.array(vectors).astype("float32")

# 6. FAISS 인덱스 생성
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# 7. 문서 매핑
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
index_to_docstore_id = {i: str(i) for i in range(len(documents))}

# 8. LangChain용 embedding 객체 (검색 시 사용)
embedding = AzureOpenAIEmbeddings(
    api_key=embedding_api_key,
    azure_endpoint=embedding_endpoint,
    model=deployment,
    openai_api_version=api_version
)

# 9. FAISS 벡터스토어 생성
vectorstore = FAISS(
    embedding_function=embedding,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)


In [91]:

# 10. 검색 예시
query = "신청기간은 어떻게돼?"
results = vectorstore.similarity_search(query, k=5)

for i, doc in enumerate(results):
    print(f"\n🔍 Top {i+1} 결과:")
    print(f"본문: {doc.page_content}")
    print(f"메타데이터: {doc.metadata}")



🔍 Top 1 결과:
본문: \* 일정은 진행상황에 따라 변동될 수 있습니다.  
모집 공고는 3월 27일(목)에 이루어지며, 신청(PC, 모바일)은 4월 7일(월)부터 4월 9일(수)까지 진행됩니다. 서류심사 대상자 발표는 4월 11일(금)에 있고, 서류 제출은 4월 14일(월)부터 4월 16일(수)까지입니다. 자격 검증은 4월 17일(목)부터 6월 12일(목)까지 이루어지며, 예비 입주자 순번 발표는 6월 13일(금)에 진행됩니다. 계약 체결(순번 도래 시)은 별도의 안내를 따르며, 입주는 계약일부터 60일 이내에 진행됩니다.
메타데이터: {'bullet': '(모집일정)'}

🔍 Top 2 결과:
본문: 신청은 시·군·구 내 주택군을 모집단위로 하고, 여러 개의 모집단위가 있는 경우 그중
한 개의 모집단위를 선택하여 신청합니다.
메타데이터: {'bullet': '(모집단위)'}

🔍 Top 3 결과:
본문: 신청 내용이 사실과 다를 경우 부적격 처리, 당첨 취소 등 불이익이 있을 수 있으며,
이로 인한 책임은 신청자 본인에게 있음을 유의하시어 공고문을 반드시 숙지하신 후 정확하게
신청하여 주시기 바랍니다.  
<!-- PageNumber="- 1 -" -->
<!-- PageBreak -->
메타데이터: {'bullet': '(신청유의)'}

🔍 Top 4 결과:
본문: 청약신청: 청약신청은 4월 7일(월) 오전 10시부터 4월 9일(수) 오후 4시까지 진행됩니다. 청년 매입임대는 인터넷(https://apply.lh.or.kr) 또는 모바일 앱(LH청약플러스)을 통해 신청 가능합니다. 공고내용을 숙지한 뒤 신청자격과 소득 등을 확인하고 신청해야 합니다. 국세청 홈택스에서 소득 조회가 가능하며, 위임장을 첨부하면 가족의 소득도 조회할 수 있습니다. 청약신청은 기간 중 24시간 가능하나 시작일과 마감일은 제외됩니다. 마감일인 4월 9일(수)의 마감시간은 오후 4시임을 유의해야 합니다. 공동인증서(개인용) 또는 민간인증서(금융인증서, 네이버인증서, 토스인증

### FAISS 라이브러리

### LLM 연동

In [95]:
import requests
import re

# Azure OpenAI GPT-4o 설정

def request_gpt(prompt: str) -> str:
    """
    GPT-4o에 프롬프트를 보내고 응답 받기
    """
    headers = {
        'Content-Type': 'application/json',
        'api-key': gpt_api_key
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": "너는 친절하고 정확한 AI 도우미야. 사용자 질문에 문서 기반으로 답해줘."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    response = requests.post(gpt_endpoint, headers=headers, json=body)

    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        content = message['content']
        # [doc1] 같은 포맷이 있을 경우 사람이 읽기 쉽게 변환
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content
    else:
        print("❌ 요청 실패:", response.status_code, response.text)
        return "⚠️ 오류가 발생했습니다."


In [96]:
def generate_answer_with_rag(query: str, vectorstore, top_k: int = 3) -> str:
    """
    FAISS에서 관련 문서를 검색한 뒤, 그 내용을 기반으로 GPT-4o에 질문하는 함수
    """
    results = vectorstore.similarity_search(query, k=top_k)

    context = "\n\n".join([f"[doc{i+1}]\n{doc.page_content}" for i, doc in enumerate(results)])

    prompt = f"""다음은 사용자가 질문한 내용과 관련된 문서 내용이야. 이 문서를 참고해서 질문에 대해 정확하고 구체적으로 답변해줘.

[사용자 질문]
{query}

[참고 문서]
{context}

답변:"""

    return request_gpt(prompt)


In [97]:
query = "임대조건 1순위는?"

# RAG 없이 일반 GPT 사용 (배경지식 기반)
basic_answer = request_gpt(query)
print("📄 GPT 일반 답변:\n", basic_answer)

# RAG 기반 답변 생성 (문서 + LLM)
rag_answer = generate_answer_with_rag(query, vectorstore)
print("📚 RAG 기반 GPT 답변:\n", rag_answer)


📄 GPT 일반 답변:
 임대주택에서 "1순위"는 일반적으로 임대주택 신청 시 우선적으로 선정될 수 있는 조건을 뜻합니다. 이는 주택의 종류(공공임대, 국민임대, 행복주택 등)에 따라 다를 수 있으며, 지역과 정책에 따라 조금씩 달라질 수 있습니다. 일반적인 기준은 다음과 같습니다:

### 공공임대주택 1순위 조건
1. **소득 기준 충족**: 가구당 월평균 소득이 일정 기준 이하(예: 도시근로자 평균 소득의 50~70% 이하)인 경우.
2. **무주택자**: 신청 당시 무주택 세대구성원이어야 합니다.
3. **자산 기준 충족**: 총 자산(부동산, 자동차 등)이 일정 금액 이하이어야 합니다.
4. **지역 거주 요건**: 해당 지역에 일정 기간 이상 거주한 경우.

### 국민임대주택 1순위 조건
1. **무주택 세대구성원**: 신청 가구 전체가 무주택이어야 합니다.
2. **소득 및 자산 기준 충족**:
   - 소득: 도시근로자 월평균 소득의 일정 비율 이하.
   - 자산: 총 자산(부동산, 자동차 등)이 일정 금액 이하.
3. **우선 순위 대상**: 장애인, 국가유공자, 한부모가족, 65세 이상 고령자, 생계·의료급여 수급자 등.

### 행복주택 1순위 조건
1. **청년, 신혼부부, 고령자 등 우선 대상**.
2. **소득 및 자산 기준 충족**.
3. **근로지 또는 학교와의 거리**: 신청자가 근로지 또는 학교와 가까운 위치에 있는 경우.

임대주택의 구체적인 조건은 주택 유형, 지역, 공급 시기에 따라 다를 수 있습니다. 정확한 기준은 **LH공사** 또는 **지역주택공사**에서 제공하는 공고문을 확인하는 것이 가장 좋습니다.
📚 RAG 기반 GPT 답변:
 임대조건에서 1순위는 다음과 같이 정의됩니다:

1. **임대조건**  
   - **임대료**: 1순위는 시중 시세의 40% 수준으로 책정됩니다.
   - **임대보증금**: 1순위의 임대보증금은 100만 원입니다.

2. **소득 및 자산 기준**  
   - 소득기준: 수급자 등